# MyScale

>[MyScale](https://docs.myscale.com/en/overview/)是基于开源[ClickHouse](https://github.com/ClickHouse/ClickHouse)构建的云数据库，专为人工智能应用和解决方案进行了优化。

本笔记本演示了如何使用与`MyScale`向量数据库相关的功能。

## 设置环境

In [ ]:
!pip install clickhouse-connect

我们想使用OpenAIEmbeddings，因此必须获取OpenAI API密钥。

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

有两种方法可以设置myscale索引的参数。

1. 环境变量

在运行应用程序之前，请使用`export`设置环境变量：
`export MYSCALE_URL='<your-endpoints-url>' MYSCALE_PORT=<your-endpoints-port> MYSCALE_USERNAME=<your-username> MYSCALE_PASSWORD=<your-password> ...`

您可以在我们的SaaS上轻松找到您的帐户，密码和其他信息。有关详细信息，请参阅[此文档](https://docs.myscale.com/en/cluster-management/)

在`MyScaleSettings`下的每个属性都可以使用前缀`MYSCALE_`进行设置，并且不区分大小写。

2. 使用参数创建`MyScaleSettings`对象


    ```python
    from langchain.vectorstores import MyScale, MyScaleSettings
    config = MyScaleSetting(host="<your-backend-url>", port=8443, ...)
    index = MyScale(embedding_function, config)
    index.add_documents(...)
    ```

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import MyScale
from langchain.document_loaders import TextLoader

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
for d in docs:
    d.metadata = {'some': 'metadata'}
docsearch = MyScale.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

Inserting data...: 100%|██████████| 42/42 [00:18<00:00,  2.21it/s]


In [5]:
print(docs[0].page_content)

As Frances Haugen, who is here with us tonight, has shown, we must hold social media platforms accountable for the national experiment they’re conducting on our children for profit. 

It’s time to strengthen privacy protections, ban targeted advertising to children, demand tech companies stop collecting personal data on our children. 

And let’s get all Americans the mental health services they need. More people they can turn to for help, and full parity between physical and mental health care. 

Third, support our veterans. 

Veterans are the best of us. 

I’ve always believed that we have a sacred obligation to equip all those we send to war and care for them and their families when they come home. 

My administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  

Our troops in Iraq and Afghanistan faced many dangers.


## 获取连接信息和数据模式

In [ ]:
print(str(docsearch))

## 过滤

您可以直接访问myscale SQL where语句。您可以按照标准SQL编写`WHERE`子句。

**注意**：请注意SQL注入，此接口不能直接由最终用户调用。

如果您在设置下自定义了`column_map`，则可以像这样使用过滤器进行搜索：

In [7]:
from langchain.vectorstores import MyScale, MyScaleSettings
from langchain.document_loaders import TextLoader

loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

for i, d in enumerate(docs):
    d.metadata = {'doc_id': i}

docsearch = MyScale.from_documents(docs, embeddings)

Inserting data...: 100%|██████████| 42/42 [00:15<00:00,  2.69it/s]


### 带有分数的相似度搜索

返回的距离分数是余弦距离。因此，较低的分数更好。

In [16]:
meta = docsearch.metadata_column
output = docsearch.similarity_search_with_relevance_scores('What did the president say about Ketanji Brown Jackson?', 
                                                           k=4, where_str=f"{meta}.doc_id<10")
for d, dist in output:
    print(dist, d.metadata, d.page_content[:20] + '...')

0.252379834651947 {'doc_id': 6, 'some': ''} And I’m taking robus...
0.25022566318511963 {'doc_id': 1, 'some': ''} Groups of citizens b...
0.2469480037689209 {'doc_id': 8, 'some': ''} And so many families...
0.2428302764892578 {'doc_id': 0, 'some': 'metadata'} As Frances Haugen, w...


## 删除您的数据

In [ ]:
docsearch.drop()